Объекты-заглушки (базы данных, API, файловая система).

## unittest.mock
Встроенный модуль для mock-тестирования

### Создание Mock-объекта
```python
from unittest.mock import Mock

mock_obj = Mock()
mock_obj.some_method.return_value = "Mocked value"

assert mock_obj.some_method() == "Mocked value"
```

### Замена реального метода

```python
def real_function():
    return "Real value"

mock_obj = Mock()
mock_obj.side_effect = real_function

assert mock_obj() == "Real value"
```


## Подмена объектов с `patch`

### Использование `patch` как декоратора
```python
from unittest.mock import patch

@patch('module_name.ClassName')
def test_func(mock_class):
    mock_instance = mock_class.return_value
    mock_instance.some_method.return_value = "Mocked result"
    
    assert some_function() == "Mocked result"
```

### Использование `patch` как контекстного менеджера
```python
from unittest.mock import patch

def test_func():
    with patch('module_name.ClassName') as mock_class:
        mock_instance = mock_class.return_value
        mock_instance.some_method.return_value = "Mocked result"
        
        assert some_function() == "Mocked result"
```


## Проверка вызовов Mock-объектов
```python
mock_obj.some_method("test")
mock_obj.some_method.assert_called_with("test")       # Проверяем вызов метода
mock_obj.some_method.assert_called_once()             # Проверяем, что метод вызван один раз
mock_obj.some_method.assert_called_once_with("test")  # Проверяем вызов с аргументами
mock_obj.some_method.assert_not_called()              # Проверяем, что метод НЕ вызывался
```

## Mock-тестирование асинхронного кода (`aiohttp`)

### Мокирование асинхронных запросов
```python
import unittest
from unittest.mock import patch, AsyncMock
import aiohttp

async def fetch_data(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.json()

class TestApi(unittest.IsolatedAsyncioTestCase):
    @patch('aiohttp.ClientSession.get', new_callable=AsyncMock)
    async def test_fetch_data(self, mock_get):
        mock_response = AsyncMock()
        mock_response.json.return_value = {"data": "mocked"}
        mock_get.return_value.__aenter__.return_value = mock_response

        result = await fetch_data("http://example.com")

        self.assertEqual(result, {"data": "mocked"})
        mock_get.assert_called_once_with("http://example.com")

if __name__ == '__main__':
    unittest.main()
```


### Использование `aioresponses`
```python
from aioresponses import aioresponses
import unittest

async def fetch_data(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.json()

class TestApi(unittest.IsolatedAsyncioTestCase):
    async def test_fetch_data(self):
        with aioresponses() as m:
            m.get("http://example.com", payload={"data": "mocked"})

            result = await fetch_data("http://example.com")

            self.assertEqual(result, {"data": "mocked"})

if __name__ == '__main__':
    unittest.main()
```


## Тестирование работы с файлами (`mock_open`)


### Мокирование чтения файла
```python
from unittest.mock import mock_open, patch

def read_file(filepath):
    with open(filepath, 'r') as f:
        return f.read()

@patch("builtins.open", new_callable=mock_open, read_data="mocked content")
def test_read_file(mock_file):
    result = read_file("dummy.txt")
    assert result == "mocked content"
    mock_file.assert_called_once_with("dummy.txt", "r")
```


### Мокирование записи в файл
```python
def write_file(filepath, content):
    with open(filepath, 'w') as f:
        f.write(content)

@patch("builtins.open", new_callable=mock_open)
def test_write_file(mock_file):
    write_file("dummy.txt", "some content")
    mock_file.assert_called_once_with("dummy.txt", "w")
    mock_file().write.assert_called_once_with("some content")
```


## Тестирование работы с базами данных (`SQLAlchemy`)
```python
from unittest.mock import patch, MagicMock

def fetch_user_by_id(session, user_id):
    return session.query(User).filter_by(id=user_id).first()

class User:
    def __init__(self, id, name):
        self.id = id
        self.name = name

@patch("your_module.session")
def test_fetch_user(mock_session):
    mock_query = MagicMock()
    mock_query.filter_by.return_value.first.return_value = User(id=1, name="John Doe")
    mock_session.query.return_value = mock_query

    result = fetch_user_by_id(mock_session, 1)

    assert result.name == "John Doe"
    mock_session.query.assert_called_once_with(User)
    mock_query.filter_by.assert_called_once_with(id=1)
```


### Тестирование транзакций
```python
def create_user(session, user):
    session.add(user)
    session.commit()

@patch("your_module.session")
def test_create_user(mock_session):
    user = User(id=1, name="John Doe")

    create_user(mock_session, user)

    mock_session.add.assert_called_once_with(user)
    mock_session.commit.assert_called_once()
```
